In [17]:
def compress(load_path, save_img_path, save_data_path, image_number):

    ######################################
    #########        BEGIN       #########
    ######### K-means clustering #########
    ######################################

    import os
    from imageio import imread
    import numpy as np
    from skimage.color import rgb2lab, lab2rgb
    from skimage.transform import rescale
    from sklearn.metrics.pairwise import euclidean_distances
    import matplotlib.pyplot as plt
    from PIL import Image

    image_raw = imread(load_path)

    image_width = 400
    image = rescale(image_raw, image_width/image_raw.shape[0], mode='reflect', channel_axis=2, anti_aliasing=True)
    shape = image.shape


    X = rgb2lab(image).reshape(-1, 3)

    def cluster_assignments(X, Y):
        return np.argmin(euclidean_distances(X,Y), axis=1)

    K = 32
    centers = np.array([X.mean(0) + (np.random.randn(3)/10) for _ in range(K)])
    y_kmeans = cluster_assignments(X, centers)

    maxreps = 5
    threshold = 1


    for b in range(maxreps):
        prev_centers = np.copy(centers)

        # assign each point to the closest center
        y_kmeans = cluster_assignments(X, centers)

        # move the centers to the mean of their assigned points (if any)
        for i, c in enumerate(centers):
            points = X[y_kmeans == i]
            if len(points):
                centers[i] = points.mean(0)
        
        #center_movement = np.mean(np.linalg.norm(centers - prev_centers, axis=1))
        max_distance = np.max(np.linalg.norm(centers - prev_centers, axis=1))

        if max_distance < threshold:
            print(f"\n\nCenter movement: {round(max_distance,3)}. Convergence achieved within", b+1, "iterations.\n\n")
            break
        elif b+1 == maxreps:
            print(f"\n\nCenter movement: {round(max_distance,3)}. Maximum repetitions reached within", b+1, "iterations.\n\n")
            break


    ######################################
    #########         END        #########
    ######### K-means clustering #########
    ######################################


    from scipy.spatial.distance import cdist

    def find_closest_centroid_color2(oldpixel, palette):
        distances = cdist([oldpixel], palette).flatten()
        closest_index = np.argmin(distances)
        return closest_index



    clustered_image = lab2rgb(centers[y_kmeans,:].reshape(shape[0], shape[1], 3))

    # Centroid colors
    #lab2rgb retrieves the rgb color in the point of the centroid given in "centers".
    #(standardized in "centers", therefore multiplied by 255 = RGB
    palette = np.uint8(lab2rgb(centers)*255)



    ######################################
    #########        BEGIN       #########
    ######### Grayscale function #########
    ######################################

    def grayscale(image):
        pil_img = Image.fromarray(np.uint8(image))
        gray_img = pil_img.convert('L')
        gray_np = np.array(gray_img)
        grayscale_img = np.stack((gray_np, gray_np, gray_np), axis=-1)
        return grayscale_img

    ######################################
    #########         END        #########
    ######### Grayscale function #########
    ######################################




    ######################################
    #########        BEGIN       #########
    #########      Dithering     #########
    ######################################


    img = image.astype(np.float64)*255  # original image (rescaled)

    original_image = Image.fromarray(np.uint8(img))
    original_image_array = np.uint8(img)

    height, width = img.shape[:2]

    for y in range(0, width):
        for x in range(0, height):
            oldpixel = np.copy(img[x, y])
            newpixel = palette[find_closest_centroid_color2(oldpixel, palette)]
            img[x, y] = newpixel
            quant_err = oldpixel - newpixel


            if x < height - 1: 
                img[x + 1, y    ] += quant_err * 7/16
            if y < width - 1: 
                img[x - 1, y + 1] += quant_err * 3/16
            if y < width - 1: 
                img[x    , y + 1] += quant_err * 5/16
            if y < width - 1 and x < height - 1: 
                img[x + 1, y + 1] += quant_err * 1/16


    # Save dithered image
    dithered_img = Image.fromarray(np.uint8(img))
    dithered_img_array = np.uint8(img)
    original_image.save(f'{save_img_path}/original_img{image_number}.bmp', format='BMP')
    dithered_img.save(f'{save_img_path}/dithered_img{image_number}.bmp', format='BMP')



    ######################################
    #########         END        #########
    #########      Dithering     #########
    ######################################



    ######################################
    #########        BEGIN       #########
    #########      Statistik     #########
    ######################################

    #####            #####
    ##### -- SSIM -- #####


    ### Dithering compression ###

    from skimage.metrics import structural_similarity as ssim
    from skimage import color

    # Convert to grayscale and cast back to uint8
    original_array = np.uint8(grayscale(original_image_array))
    dithered_array = np.uint8(grayscale(dithered_img_array))

    # Calculate SSIM
    ssim1, _ = ssim(original_array, dithered_array, full=True, channel_axis=2)

    print(f"SSIM Index: {round(ssim1, 3)}")




    ### JPEG compression ###

    # Load the images using Pillow
    uncompressed_img = Image.fromarray(np.uint8(image*255))
    uncompressed_img.save(f'{save_img_path}/jpg_img{image_number}.jpg', "JPEG", optimize = True, quality = 1) 

    compressed_img = Image.open(f'{save_img_path}/jpg_img{image_number}.jpg')
    
    # Convert Pillow images to NumPy arrays
    original_array = np.uint8(grayscale(original_image_array))
    compressed_array = np.uint8(grayscale(np.array(compressed_img)))

    # Calculate SSIM
    ssim2, _ = ssim(original_array, compressed_array, full=True, channel_axis=2)

    print(f"SSIM Index: {round(ssim2, 3)}")




    #####           #####
    ##### -- MSE -- #####
    from skimage.metrics import mean_squared_error

    mse1 = mean_squared_error(original_image_array, dithered_img_array)

    mse2 = mean_squared_error(original_image_array, np.array(compressed_img))

    print("Original / dithered")
    print("MSE:", round(mse1,2))

    print("\nOriginal / JPG-compressed")
    print("MSE:", round(mse2,2))

    print("\n\nControl with same image: \n", mean_squared_error(original_image_array, original_image_array))


    #####            #####
    ##### -- PSNR -- #####

    from skimage.metrics import peak_signal_noise_ratio

    psnr1 = peak_signal_noise_ratio(original_image_array, dithered_img_array)
    psnr2 = peak_signal_noise_ratio(original_image_array, np.array(compressed_img))


    print("Original / dithered")
    print(f"PSNR: {round(psnr1,2)} dB")

    print("\nOriginal / JPG-compressed")
    print(f"PSNR: {round(psnr2, 2)} dB")

    print("\n\nControl with same image: \n",peak_signal_noise_ratio(original_image_array, original_image_array))


    #####             #####
    ##### -- LPIPS -- #####


    ## Dithered ##
    from piq import LPIPS
    from torchvision.transforms.functional import to_tensor

    # Convert numpy arrays to PyTorch tensors
    original_tensor = to_tensor(original_image_array).unsqueeze(0)
    dithered_tensor = to_tensor(dithered_img_array).unsqueeze(0)

    # Calculate LPIPS
    lpips1 = LPIPS()(original_tensor, dithered_tensor)
    print(f"LPIPS Loss: {round(lpips1.item(), 3)}")



    ## JPEG ##
    from piq import LPIPS
    from torchvision.transforms.functional import to_tensor

    # Convert numpy arrays to PyTorch tensors
    original_tensor = to_tensor(original_image_array).unsqueeze(0)
    dithered_tensor = to_tensor(dithered_img_array).unsqueeze(0)

    # Calculate LPIPS
    lpips2 = LPIPS()(original_tensor, dithered_tensor)
    print(f"LPIPS Loss: {round(lpips2.item(), 3)}")

    

    ### Save data in a .txt file
    ssim1, ssim2, mse1, mse2, psnr1, psnr2, lpips1, lpips2 = (
        round(ssim1, 3), round(ssim2, 3), round(mse1, 3), round(mse2, 3),
        round(psnr1, 3), round(psnr2, 3), round(lpips1.item(), 3), round(lpips2.item(), 3)
    )
    with open(save_data_path, "a") as file:
        file.write(f"{ssim1};{ssim2};{mse1};{mse2};{psnr1};{psnr2};{lpips1};{lpips2}\n")




In [18]:
import os

def count_files(load_path):
    files = [f for f in os.listdir(load_path) if os.path.isfile(os.path.join(load_path, f))]
    num_files = len(files)
    return num_files

In [19]:
uncompressed_path = f'C:/Users/Lenovo/Desktop/DTU/1. semester/Introduktion til Intelligente Systemer/Eksamen/3-ugers projekt/data/uncompressed/'

save_data_path = r'C:\Users\Lenovo\Desktop\DTU\1. semester\Introduktion til Intelligente Systemer\Eksamen\3-ugers projekt\data\data.txt'
#range(count_files(load_path))
for i in range(2):
    uncompressed_images = [f for f in os.listdir(uncompressed_path) if os.path.isfile(os.path.join(uncompressed_path, f))]
    load_path = f'C:/Users/Lenovo/Desktop/DTU/1. semester/Introduktion til Intelligente Systemer/Eksamen/3-ugers projekt/data/uncompressed/{uncompressed_images[i]}'
    save_img_path = f'C:/Users/Lenovo/Desktop/DTU/1. semester/Introduktion til Intelligente Systemer/Eksamen/3-ugers projekt/data/compressed/img{i}'
    folder_path = os.makedirs(save_img_path, exist_ok=True)
    compress(load_path, save_img_path, save_data_path, i)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16540\1538109023.py:17: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_raw = imread(load_path)




Center movement: 2.018. Maximum repetitions reached within 5 iterations.


SSIM Index: 0.699
SSIM Index: 0.55
Original / dithered
MSE: 222.04

Original / JPG-compressed
MSE: 416.39


Control with same image: 
 0.0
Original / dithered
PSNR: 24.67 dB

Original / JPG-compressed
PSNR: 21.94 dB


Control with same image: 
 inf


c:\Users\Lenovo\Desktop\DTU\1. semester\Introduktion til Intelligente Systemer\Eksamen\3-ugers projekt\3week24_venv\Lib\site-packages\skimage\metrics\simple_metrics.py:163: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range ** 2) / err)
c:\Users\Lenovo\Desktop\DTU\1. semester\Introduktion til Intelligente Systemer\Eksamen\3-ugers projekt\3week24_venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\DTU\1. semester\Introduktion til Intelligente Systemer\Eksamen\3-ugers projekt\3week24_venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `we

LPIPS Loss: 0.494
LPIPS Loss: 0.494


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16540\1538109023.py:17: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_raw = imread(load_path)




Center movement: 4.85. Maximum repetitions reached within 5 iterations.


SSIM Index: 0.731
SSIM Index: 0.575
Original / dithered
MSE: 204.83

Original / JPG-compressed
MSE: 337.41


Control with same image: 
 0.0
Original / dithered
PSNR: 25.02 dB

Original / JPG-compressed
PSNR: 22.85 dB


c:\Users\Lenovo\Desktop\DTU\1. semester\Introduktion til Intelligente Systemer\Eksamen\3-ugers projekt\3week24_venv\Lib\site-packages\skimage\metrics\simple_metrics.py:163: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range ** 2) / err)
c:\Users\Lenovo\Desktop\DTU\1. semester\Introduktion til Intelligente Systemer\Eksamen\3-ugers projekt\3week24_venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\Desktop\DTU\1. semester\Introduktion til Intelligente Systemer\Eksamen\3-ugers projekt\3week24_venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `we



Control with same image: 
 inf
LPIPS Loss: 0.436
LPIPS Loss: 0.436
